In [6]:
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
from tba_types import Match, Event, SimpleMatch
from tba_session import session, ROOT_URL
from collections import defaultdict

In [7]:
START_YEAR = 2013
END_YEAR = 2025

plt.rcParams['figure.figsize'] = (20, 10)

In [19]:
blue_winrates = defaultdict(float)
red_winrates = defaultdict(float)

for year in trange(START_YEAR, END_YEAR + 1, unit="year"):
    if year == 2015 or year == 2021: continue
    events: list[Event] = session.get(
        f"{ROOT_URL}/events/{year}"
    ).json()
    idx = year - START_YEAR
    for event in tqdm(events, unit="event", leave=False):
        if not 0 <= event["event_type"] <= 5:
            continue
        matches: list[SimpleMatch] = session.get(
            f"{ROOT_URL}/event/{event["key"]}/matches/simple"
        ).json()
        red_wins = 0
        blue_wins = 0
        ties = 0
        for match in filter(lambda m: m["comp_level"] == "qm", matches):
            if match["winning_alliance"] == "blue":
                blue_wins += 1
            elif match["winning_alliance"] == "red":
                red_wins += 1
            else:
                ties += 1
        total = red_wins + blue_wins + ties
        if total == 0: continue
        blue_winrates[event["key"]] = blue_wins / total
        red_winrates[event["key"]] = red_wins / total

  0%|          | 0/13 [00:00<?, ?year/s]

  0%|          | 0/129 [00:00<?, ?event/s]

  0%|          | 0/166 [00:00<?, ?event/s]

  0%|          | 0/203 [00:00<?, ?event/s]

  0%|          | 0/256 [00:00<?, ?event/s]

...

  0%|          | 0/196 [00:00<?, ?event/s]

  0%|          | 0/288 [00:00<?, ?event/s]

  0%|          | 0/309 [00:00<?, ?event/s]

  0%|          | 0/323 [00:00<?, ?event/s]

  0%|          | 0/222 [00:00<?, ?event/s]

In [24]:
for event, blue_winrate, red_winrate in sorted(zip(
    blue_winrates.keys(), blue_winrates.values(), red_winrates.values()
), key=lambda x: abs(x[1] - x[2]), reverse=True)[:20]:
    print(
        f"Event {event} has a blue winrate of {blue_winrate*100:.2f}% and a red winrate of {red_winrate*100:.2f}%"
    )

Event 2022on409 has a blue winrate of 20.83% and a red winrate of 79.17%
Event 2022va319 has a blue winrate of 70.59% and a red winrate of 14.71%
Event 2022dc320 has a blue winrate of 66.67% and a red winrate of 29.63%
Event 2022dc312 has a blue winrate of 33.33% and a red winrate of 66.67%
Event 2022on204 has a blue winrate of 66.67% and a red winrate of 33.33%
Event 2017txda has a blue winrate of 31.94% and a red winrate of 63.89%
Event 2017ncash has a blue winrate of 33.33% and a red winrate of 64.81%
Event 2017onto1 has a blue winrate of 65.38% and a red winrate of 34.62%
Event 2017mitry has a blue winrate of 64.63% and a red winrate of 35.37%
Event 2016flwp has a blue winrate of 63.33% and a red winrate of 34.44%
Event 2017miliv has a blue winrate of 63.41% and a red winrate of 35.37%
Event 2018vabla has a blue winrate of 62.90% and a red winrate of 35.48%
Event 2020txcha has a blue winrate of 35.48% and a red winrate of 62.90%
Event 2017casd has a blue winrate of 61.36% and a red

In [9]:
TEAM = "frc1986"
YEAR = 2017

team_events: list[Event] = session.get(
    f"{ROOT_URL}/team/{TEAM}/events"
).json()

blue_matches = 0
blue_wins = 0
blue_auto = 0
red_matches = 0
red_wins = 0
red_auto = 0

for event in filter(lambda e: e["year"] == YEAR and 0 <= e["event_type"] <= 5, team_events):
    matches: list[Match] = session.get(
        f"{ROOT_URL}/team/{TEAM}/event/{event["key"]}/matches"
    ).json()
    for match in matches:
        if TEAM in match["alliances"]["blue"]["team_keys"]:
            blue_matches += 1
            blue_wins += match["winning_alliance"] == "blue"
            blue_auto += match["score_breakdown"]["blue"]["autoPoints"]
            breakdown = match["score_breakdown"]["blue"]
        elif TEAM in match["alliances"]["red"]["team_keys"]:
            red_matches += 1
            red_wins += match["winning_alliance"] == "red"
            red_auto += match["score_breakdown"]["red"]["autoPoints"]

print(f"{TEAM[3:]} in {YEAR}:")
print(f"Overall: {(red_wins + blue_wins) / (red_matches + blue_matches) * 100:.2f}% winrate ({red_wins + blue_wins}/{red_matches + blue_matches}), {(red_auto + blue_auto) / (red_matches + blue_matches):.1f} average auto")
print(f"Red: {red_wins / red_matches * 100:.2f}% winrate ({red_wins}/{red_matches}), {red_auto / red_matches:.1f} average auto")
print(f"Blue: {blue_wins / blue_matches * 100:.2f}% winrate ({blue_wins}/{blue_matches}), {blue_auto / blue_matches:.1f} average auto")

1986 in 2017:
Overall: 80.88% winrate (55/68), 92.8 average auto
Red: 77.27% winrate (34/44), 99.1 average auto
Blue: 87.50% winrate (21/24), 81.3 average auto
